# 13. 동적 헤밀턴 몬테카를로, No-U-Turn 샘플링 (Dynamic Hamiltonian Monte Carlo, No-U-Turn Sampling (NUTS))
- HMC에서 해밀토니안 시스템을 시뮬레이션하는 것은 단봉 확률 분포를 탐색하는 데 매우 효율적인 방법을 제공
- 해밀토니안 역학의 에너지 보존 특성 덕분에, 타겟 공간에서 임의로 긴 궤적을 시뮬레이션할 수 있으며 이에 따른 제안들은 본질적으로 100%에 가까운 확률로 수용

- **샘플링 측면에서 표준 HMC의 가장 큰 미해결 문제 중 하나는 해밀토니안 시스템 시뮬레이션을 위한 스텝 수 $L$의 선택**

    - 스텝 수가 많을수록 HMC는 MCMC 체인에 대해 더 긴 이동을 제안함으로써 더 큰 이점을 제공
    - **그러나 $L$이 너무 크면 궤적이 다시 돌아오는 문제가 발생,** 
        
        **이는 결국 시작점으로 되돌아와 버려서 불필요한 계산만 많이 하게 될 위험**
        
    **즉, $L$을 과도하게 크게 선택하면 타겟 밀도 주변에서 긴 궤적을 시뮬레이션하는 데 많은 계산 자원을 소모한 후에도,**
    
    **결과적으로 매우 짧은 MCMC 이동만 제안되는 상황에 빠질 수 있음**

<br>

- 또 다른 문제는 표준 HMC에서 해밀토니안 시스템의 시뮬레이션이 '올인(all-or-nothing)' 방식이라는 점
    
    - **만약 시뮬레이션의 마지막 단계에서 얻은 최종 상태가 MH 단계에서 거부된다면(예를 들어, 마지막 스텝이 시뮬레이션을 부적절한 영역으로 이끌었다면)** 
        
        **그동안 수행한 모든 계산이 허사가 되어 해밀토니안 시뮬레이션을 처음부터 다시 시작함.**
        
    $\rightarrow$ **이러한 문제들을 해결하기 위해 동적 HMC(dynamic HMC) 방법이 고정된 스텝 수 $L$을 선택하는 개념을 버리고,**
        
    **대신 시뮬레이션된 궤적에 따라 MCMC 체인의 다음 점을 동적으로 선택**

<br>

## 13-01.  No-U-Turn Sampling (NUTS)
- Hoffman과 Gelman (2014)의 No-U-Turn Sampler (NUTS)는 최초로 제안된 동적 HMC 알고리즘
- 직관적으로 NUTS는 시뮬레이션된 궤적을 반복적으로 확장하여, 경로가 자기 자신으로 돌아서 U턴을 형성하는 시점을 감지할 때까지 진행
    
    가역성을 보장하기 위해, 이 경로는 가상 시간에서 앞으로와 뒤로 모두 확장
    
- 또한 단일 수용/거부 결정 대신, 알고리즘은 경로를 따라 생성된 후보 집합 중에서 다음 점을 샘플링
    
- 이 알고리즘은 $\epsilon$의 튜닝을 위한 휴리스틱도 포함

<br>

## 13-02.  NUTS $\epsilon$ 튜닝
- 일반적으로, $\epsilon$ 값이 너무 크면 불안정한 leapfrog 적분을 초래하고, 너무 작으면 계산 자원이 낭비

- NUTS에서는 먼저 $\epsilon$ 값을 두 배 또는 절반으로 조정하여, Langevin 제안의 수용률이 약 0.5가 되는 $\epsilon$을 탐색.
- 이 초기 추정치는 워밍업(iteration) 동안 원하는 목표 수용률(예: 0.8)을 달성하도록 추가 최적화

<br>

## 13-03.  NUTS 샘플링
- NUTS의 샘플링 반복은 HMC와 두 가지 측면에서 차이:
1. 해밀토니안 시스템에서 시뮬레이션된 궤적(또는 경로)이 U턴을 감지하기 위한 종료 조건이 충족될 때까지 동적으로 앞으로와 뒤로 확장
2. 단일 수용/거부 결정 대신, 위 확장 단계에서 생성된 유효한 후보들 중에서 다음 점이 샘플링


<br>

## 13-04. 경로 확장
- 단계 $j$에서는 시뮬레이션 방향(시간적으로 앞으로 또는 뒤로)이 무작위로 선택되고, 
    
    해당 경로의 끝에 $2^{j-1}$개의 leapfrog 스텝이 추가
- 평가된 상태들은 그림과 같이 이진 탐색 트리에 저장
    - 이러한 경로를 앞뒤로 확장하는 것은 과정의 가역성을 보장하기 위해 반드시 필요
    
<img src='https://www.cs.helsinki.fi/u/ahonkela/teaching/compstats1/book/images/nuts_path_extension.png'>

- 경로 확장 과정은 U턴을 감지하기 위한 종료 조건이 충족될 때까지 계속
    - 이 조건은 현재 제안된 값 $\tilde{\theta}$와 시작점 또는 초기값 $\theta$를 비교하는 데 기반
    
    - 이 두 점 사이의 제곱 거리의 도함수는
    
    $\frac{\,\mathrm{d}}{\,\mathrm{d} t} \frac{(\tilde{\mathbf{\theta}} - \mathbf{\theta}) \cdot (\tilde{\mathbf{\theta}} - \mathbf{\theta})}{2} = (\tilde{\mathbf{\theta}} - \mathbf{\theta}) \cdot \frac{\,\mathrm{d}}{\,\mathrm{d} t}(\tilde{\mathbf{\theta}} - \mathbf{\theta}) = (\tilde{\mathbf{\theta}} - \mathbf{\theta}) \cdot \tilde{\mathbf{r}},$
    
    $\tilde{\mathbf{r}}$은 현재의 운동량

<br>

- 마지막 항목이 음수가 되는 시점을 모니터링하면, 두 점 사이의 거리가 감소하기 시작하는, 
    
    즉 시뮬레이션이 시작점으로 되돌아가기 시작하는 시점 (그림의 궤적 끝부분에 화살표)을 알 수 있음

<img src='https://www.cs.helsinki.fi/u/ahonkela/teaching/compstats1/book/images/nuts_sample_selection.png'>

<br>

## 13-05. 샘플 선택
- NUTS의 또 다른 중요한 혁신은 다음 샘플을 선택하는 방식
    - 이 과정은 결과적으로 생성되는 샘플러가 상세 균형(detailed balance)을 만족하도록 신중하게 설계되어야 함
    
    - MH수용 검사를 사용하지 않기 때문에, 이 부분은 별도로 검증되고 증명되어야 함

- Hoffman과 Gelman (2014)의 원래 NUTS 알고리즘은 슬라이스 샘플링(slice sampling)으로 알려진 방법을 기반
    - 이 방법에서는 효과적으로 먼저 거부 에너지 임계값
    
        $u \sim \mathrm{Uniform}(0, \exp(- H(\mathbf{\theta}, \mathbf{r}))).$을 샘플링
    
    - $u$의 의미를 이해하기 위해, 
        
        HMC에 대한 MH수용확률 $\log a = H(\mathbf{\theta}, \mathbf{r}) - H(\mathbf{\theta}', \mathbf{r}').$에서, 수용검사는 $v \sim \mathrm{Uniform}(0, 1)$을 시뮬레이션하고,
        
        $\log v < H(\mathbf{\theta}, \mathbf{r}) - H(\mathbf{\theta}', \mathbf{r}').$를 검사함으로써 이루어짐
        
        이를 지수화하면,  $v \exp(- H(\mathbf{\theta}, \mathbf{r})) < \exp(- H(\mathbf{\theta}', \mathbf{r}')).$와 같이 표현할 수 있으며,
        
        좌변은 $u$와 동일하므로, 제안된 상태가 수용되기 위한 조건은 
        
        $u < \exp (-H(\theta', r'))$임을 의미
        
    - NUTS에서는 이 일반적인 검사를 확장하며, 단일 상태를 선택하여 임계값을 통과하는지를 확인하는 대신, 임계값을 통과하는 여러 후보들을 고려하고, 
        
        최종 샘플은 이들 유효 후보들 중에서 균등하게 샘플링하여 선택
        
        - 후보 선택에는 Hoffman과 Gelman (2014)에서 제시한 추가 조건들이 포함